In [1]:
#Diccionario
#i=inlet
#fn=fan
#c1=compressor1/LPC
#c2=compressor2/HPC
#t1=turbine1/HPT
#t2=turbine2/LPT
#n=nozzle
#mf=gasto_combustible
#M=Mach
#t=temperatura_ambiente
#p=presión_ambiente
#bpr=bypass_ratio
#cp=calor_presión_constante
#PI_fn=pressure_index_fan
#PI_c1=pressure_index_compressor1
#PI_c2=pressure_index_compressor2
#PI_cc=pressure_index_combustion_chamber
#eta_fn=rendimiento_fan
#eta_c1=rendimiento_compressor1
#eta_c2=rendimiento_compressor2
#eta_cc=rendimiento_combustion_chamber
#eta_t1=rendimiento_turbine2
#eta_t2=rendimiento_turbine2
#eta_n=rendimiento_nozzle
#eta_gearbox=rendimiento_gearbox

In [2]:
# Motivos sin extracción de datos
# Casos múltiples
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import math
import random
import datetime
import csv
from datetime import datetime
from huracan.engine import shaft
from huracan.thermo.fluids import gas, fuel
from huracan.components import inlet, fan, compressor, combustion_chamber, turbine, nozzle

# Nombre del archivo Excel con la fecha actual
fecha_actual = datetime.now().strftime('%Y-%m-%d')
nombre_archivo_in = f'datos_{fecha_actual}_in.csv'
nombre_archivo_out = f'datos_{fecha_actual}_out.csv'
nombre_archivo_marks = f'datos_{fecha_actual}_marks.csv'
nombre_archivo_marks_in = f'datos_{fecha_actual}_marks_in.csv'
nombre_archivo_marks_out = f'datos_{fecha_actual}_marks_out.csv'
nombre_archivo_marks_norm = f'datos_{fecha_actual}_marks_norm.csv'
nombre_archivo_ia = f'datos_{fecha_actual}_ia.csv'
nombre_archivo_ia_clasif = f'datos_{fecha_actual}_ia_clasif.csv'

# Ruta de la carpeta
carpeta = os.getcwd()
# carpeta = r'C:\Users\anune\Documents\MÁSTER UEM\TFM\tfm motor\huracan-master'

# Combinar la ruta de la carpeta con el nombre del archivo
ruta_completa_in = os.path.join(carpeta, nombre_archivo_in)
ruta_completa_out = os.path.join(carpeta, nombre_archivo_out)
ruta_completa_marks = os.path.join(carpeta, nombre_archivo_marks)
ruta_completa_marks_in = os.path.join(carpeta, nombre_archivo_marks_in)
ruta_completa_marks_out = os.path.join(carpeta, nombre_archivo_marks_out)
ruta_completa_marks_norm = os.path.join(carpeta, nombre_archivo_marks_norm)
ruta_completa_ia = os.path.join(carpeta, nombre_archivo_ia)
ruta_completa_ia_clasif = os.path.join(carpeta, nombre_archivo_ia_clasif)

# Número de iteraciones
num_iteraciones = 1000

# Rango de variación
variacion_base = 0.05 # Variación base para parámetros 1 < x < 100000 -> 5%
variacion_minima = 0.01  # Variación mínima para parámetros x < 1 -> 1%
variacion_maxima = 0.1  # Variación máxima para parámetros x > 100000 -> 10%

# Calificación iteraciones
def calcular_puntuacion(parametro, valor_iteracion, valor_optimo, rango_min, rango_max):
    # Calcula la puntuación de un parámetro en una iteración en función de su cercanía al valor óptimo.
    if valor_iteracion < rango_min or valor_iteracion > rango_max:
        return 0  # Valor fuera de rango, puntuación mínima (0)
    else:
        diferencia = abs(valor_optimo - valor_iteracion)
        rango_valores = rango_max - rango_min
        proporcion = diferencia / rango_valores
        puntuacion = 10 - (proporcion * 10)
        puntuacion = max(1, round(puntuacion, 0))  # La puntuación mínima es 1
        return puntuacion

# Crear función para normalizar las puntuaciones del 1 al 10 de las iteraciones
def escalar_puntuaciones(puntuaciones):
    # Encontrar el valor mínimo y máximo de las puntuaciones
    min_puntuacion = min(puntuaciones)
    max_puntuacion = max(puntuaciones)
    # Calcular el rango de las puntuaciones
    rango_puntuaciones = max_puntuacion - min_puntuacion
    # Crear una lista para almacenar las nuevas puntuaciones escaladas
    puntuaciones_escaladas = []
    # Bucle de las iteraciones
    for puntuacion in puntuaciones:
        # Aplicar la fórmula de escalado min-max
        nueva_puntuacion = ((puntuacion - min_puntuacion) / rango_puntuaciones) * 9 + 1
        # Redondear la nueva puntuación al entero más cercano
        puntuacion_escalada = round(nueva_puntuacion, 2)
        puntuaciones_escaladas.append(puntuacion_escalada)
    return puntuaciones_escaladas

#Crear una lista para almacenar todas las puntuaciones de cada iteración y sus parámetros
puntuaciones_parametros = {}
puntuaciones_parametros_out = {}
puntuaciones_iteraciones = []
puntuaciones_iteraciones_out = []
puntuaciones_iteraciones_TOTAL = []
peores_parametros_puntuaciones_por_iteracion = []
puntuaciones_parametros_in_norm = []
puntuaciones_parametros_out_norm = []

# Define los parámetros fijos
eta_gearbox=0.975

# Define los valores del caso de prueba
variables = {'mf': 1440,
             'M': 0.4,
             't': 281.65,
             'p': 89874,
             'bpr': 9.6,
             'PI_i': 0.98,
             'eta_fn': 0.92,
             'PI_fn': 1.54,
             'eta_c1': 0.89,
             'PI_c1': 1.54,
             'eta_c2': 0.89,
             'PI_c2': 9.86,
             'eta_cc': 0.965,
             'PI_cc': 0.98,
             'eta_t1': 0.89,
             'eta_t2': 0.89,
             'eta_n': 0.95,
             'LHV': 43e6}

# Valores óptimos de los parámetros
valores_optimos = {'t': (300, 250, 350),
                   'p': (101325, 85000, 130000),
                   'bpr': (9, 5, 12),
                   'eta_fn': (0.9, 0.80, 1.0),
                   'eta_c1': (0.95, 0.80, 1.0),
                   'eta_c2': (0.95, 0.80, 1.0),
                   'eta_cc': (0.98, 0.80, 1.1),
                   'eta_t1': (0.96, 0.80, 1.0),
                   'eta_t2': (0.96, 0.80, 1.0),
                   'eta_n':(0.95, 0.80, 1.1),
                   'mf': (1500, 1200, 1800),
                   'PI_i':(1.25, 0.5, 2),
                   'PI_fn': (1.25, 0.5, 2),
                   'PI_c1': (5, 1, 10),
                   'PI_c2': (8.5, 5, 12),
                   'PI_cc': (1.03, 0.9, 1.1),
                   'M': (0.8, 0, 0.85),
                   'LHV': (44e6, 30e6, 60e6),
                   'eta_gearbox': (0.975, 0.95, 1)}

valores_optimos_out = {'i.t0': (300, 250, 350),
                       'i.p0': (95000, 80000, 120000),
                       'fn.t0': (300, 250, 375),
                       'fn.p0': (150000, 100000, 200000),
                       'c1.t0': (450, 300, 600),
                       'c1.p0': (300000, 190000, 400000),
                       'c2.t0': (850, 600, 900),
                       'c2.p0': (2250000, 1500000, 3000000),
                       'cc.t0': (1500, 1200, 1900),
                       'cc.p0': (2250000, 1500000, 3000000),
                       't1.t0': (1250, 900, 1500),
                       't1.p0': (600000, 400000, 850000),
                       't2.t0': (850, 600, 1000),
                       't2.p0': (70000, 30000, 100000),
                       'n.t0': (450, 300, 600),
                       'n.p0': (150000, 100000, 200000)}

# Crear dataframes vacíos dondo guardar los valores de las iteraciones
df1_aux = pd.DataFrame()
df2_aux = pd.DataFrame()
puntuaciones_in_df_aux = pd.DataFrame()
puntuaciones_out_df_aux = pd.DataFrame()
puntuaciones_parametros_in_norm_df = pd.DataFrame()
puntuaciones_parametros_out_norm_df = pd.DataFrame()
puntuaciones_parametros_in_norm_df_aux = pd.DataFrame()
puntuaciones_parametros_out_norm_df_aux = pd.DataFrame()
df_ia = pd.DataFrame()

# Iterar sobre cada variable de entrada y actualizar su valor
for iteracion in range(num_iteraciones):
    print(f'Iteración {iteracion+1}:')
    variables_random = dict()
    for var in variables:
        # Determinar la variación aleatoria según el valor del parámetro dentro del rango especificado
        if variables[var] < 1:
            variacion = variacion_minima
        if variables[var] > 100000:
            variacion = variacion_maxima 
        else:
            variacion = variacion_base
        cambio_porcentual_aleatorio = random.uniform(-variacion, variacion)
        # Aplicar la variación aleatoria al valor actual
        variable_random = variables[var] * (1 + cambio_porcentual_aleatorio)           
        # Mostrar las variables actualizadas
        print(f'{var}: {variable_random}')
        # Constuir un diccionario
        variables_random[var] = variable_random
        #Obtener los valores óptimos para el parámetro actual
        valor_optimo, rango_min, rango_max = valores_optimos[var]
        # Calcular la puntuación para el parámetro actual
        puntuacion_parametro = calcular_puntuacion(var, variable_random, valor_optimo, rango_min, rango_max)
        puntuaciones_parametros[var] = puntuacion_parametro
    
    print(puntuaciones_parametros)  
    print(variables_random)
    
    # Crear instancias con los valores de las variables actualizadas
    f = fuel(LHV=round(variables_random['LHV'],0))       
    g = gas(mf=variables_random['mf'],
            cp=lambda T: 1150 if T > 600 else 1000,
            k=lambda T: 1.33 if T > 600 else 1.4,
            m=variables_random['M'],
            t_0=variables_random['t'],
            p_0=variables_random['p'])       
    i = inlet(PI=variables_random['PI_i'])
    fn = fan(eta=variables_random['eta_fn'], PI=variables_random['PI_fn'])
    c1 = compressor(eta=variables_random['eta_c1'], PI=variables_random['PI_c1'])
    c2 = compressor(eta=variables_random['eta_c2'], PI=variables_random['PI_c2'])
    cc = combustion_chamber(fuel=f, eta=variables_random['eta_cc'], PI=variables_random['PI_cc'])
    t1 = turbine(eta=variables_random['eta_t1'])
    t2 = turbine(eta=variables_random['eta_t2'])
    n = nozzle(eta=variables_random['eta_n'])
    shaft1 = shaft(fn, c1, t2, eta=variables_random['eta_c1'], eta_gearbox=eta_gearbox)
    shaft2 = shaft(c2, t1, eta=variables_random['eta_c2'])
    stream = g - i - fn
    s1core, s1bypass = stream * (variables_random['bpr'] / (variables_random['bpr'] + 1))
    s1core - c1 - c2 - cc - t1 - t2
    s2 = s1core - s1bypass
    s2 - n
    stream.run()

    # Crear los DataFrames
    datos_in = [variables_random['mf'], 
                variables_random['M'], 
                variables_random['t'], 
                variables_random['p'], 
                variables_random['LHV'], 
                variables_random['bpr'], 
                variables_random['eta_fn'], 
                variables_random['eta_c1'], 
                variables_random['eta_c2'], 
                variables_random['eta_t1'], 
                variables_random['eta_t2'], 
                variables_random['eta_n'], 
                variables_random['eta_cc'], 
                variables_random['PI_i'], 
                variables_random['PI_fn'], 
                variables_random['PI_c1'], 
                variables_random['PI_c2'], 
                variables_random['PI_cc']]
    
    datos_out = [i.t0, i.p0, fn.t0, fn.p0, c1.t0, c1.p0, c2.t0, c2.p0, cc.t0, cc.p0, t1.t0, t1.p0, t2.t0, t2.p0, n.t0, n.p0]
    
    datos_in = [tuple(datos_in)]
    datos_out = [tuple(datos_out)]
    
    valores_out = {'i.t0':i.t0,
                   'i.p0':i.p0,
                   'fn.t0':fn.t0,
                   'fn.p0':fn.p0,
                   'c1.t0':c1.t0,
                   'c1.p0':c1.p0,
                   'c2.t0':c2.t0,
                   'c2.p0':c2.p0,
                   'cc.t0':cc.t0,
                   'cc.p0':cc.p0,
                   't1.t0':t1.t0,
                   't1.p0':t1.p0,
                   't2.t0':t2.t0,
                   't2.p0':t2.p0,
                   'n.t0':n.t0,
                   'n.p0':n.p0}
    
    # Obtener puntuaciones parámetros de salida
    for var_out in valores_out.keys():
        #Obtener los valores óptimos para el parámetro OUT actual
        valor_optimo_out, rango_min_out, rango_max_out = valores_optimos_out[var_out]
        # Calcular la puntuación para el parámetro OUT actual
        puntuacion_parametro_out = calcular_puntuacion(var_out, valores_out[var_out], valor_optimo_out, rango_min_out, rango_max_out)
        puntuaciones_parametros_out[var_out] = puntuacion_parametro_out
    
    # Obtener valores de salida
    print(puntuaciones_parametros_out)
    print(valores_out)
    
    # Encontrar el parámetro asociado a la peor puntuación
    todas_puntuaciones = {**puntuaciones_parametros, **puntuaciones_parametros_out}
    peor_parametro = min(todas_puntuaciones, key=todas_puntuaciones.get)
    peor_puntuacion = todas_puntuaciones[peor_parametro]
    peores_parametros_puntuaciones_por_iteracion.append((peor_parametro, peor_puntuacion))
    
    # Crear un DataFrame con los datos de salida
    nombres_columnas_out = ['T0_inlet [K]', 
                            'p0_inlet [Pa]',
                            'T0_fan [K]', 
                            'p0_fan [Pa]',
                            'T0_compressor1 [K]', 
                            'p0_compressor1 [Pa]',
                            'T0_compressor2 [K]', 
                            'p0_compressor2 [Pa]', 
                            'T0_combustionchamber [K]', 
                            'p0_combustionchamber [Pa]', 
                            'T0_turbine1 [K]', 
                            'p0_turbine1 [Pa]',
                            'T0_turbine2 [K]', 
                            'p0_turbine2 [Pa]',
                            'T0_nozzle [K]', 
                            'p0_nozzle [Pa]']
    
    df1 = pd.DataFrame(datos_out, columns=nombres_columnas_out)
    
    # Crear un DataFrame con los datos de entrada
    nombres_columnas_in = ['mf', 
                           'M', 
                           't', 
                           'p', 
                           'LHV', 
                           'bpr', 
                           'eta_fn', 
                           'eta_c1', 
                           'eta_c2', 
                           'eta_t1', 
                           'eta_t2', 
                           'eta_n', 
                           'eta_cc', 
                           'PI_i', 
                           'PI_fn', 
                           'PI_c1', 
                           'PI_c2', 
                           'PI_cc']
    
    df2 = pd.DataFrame(datos_in, columns=nombres_columnas_in)
    
    # Calcular las puntuaciones de cada parámetro y la puntuación total promedio de la iteración
    if 0 in puntuaciones_parametros.values():
        puntuaciones_iteraciones.append(0)
    else:
        puntuacion_iteracion = round(sum(puntuaciones_parametros.values()) / len(puntuaciones_parametros),3)
        print('Puntuación iteración IN: ',puntuacion_iteracion)
        puntuaciones_iteraciones.append(puntuacion_iteracion)
    
    if 0 in puntuaciones_parametros_out.values():
        puntuaciones_iteraciones_out.append(0)
    else:
        puntuacion_iteracion_out = round(sum(puntuaciones_parametros_out.values()) / len(puntuaciones_parametros_out),3)
        print('Puntuación iteración OUT: ',puntuacion_iteracion_out)
        puntuaciones_iteraciones_out.append(puntuacion_iteracion_out)
    
    if 0 in puntuaciones_parametros.values() or 0 in puntuaciones_parametros_out.values():
        puntuaciones_iteraciones_TOTAL.append(0)
    else:
        suma_puntuaciones_parametros_in = sum(puntuaciones_parametros.values())
        suma_puntuaciones_parametros_out = sum(puntuaciones_parametros_out.values())
        num_parametros = len(puntuaciones_parametros) + len(puntuaciones_parametros_out)
        puntuacion_iteracion_TOTAL = round((suma_puntuaciones_parametros_in + suma_puntuaciones_parametros_out) / num_parametros,3)
        puntuaciones_iteraciones_TOTAL.append(puntuacion_iteracion_TOTAL)
    
    # Normalizar las puntuaciones de entrada y salida para esta iteración
    puntuaciones_parametros_out_norm_iter = escalar_puntuaciones(puntuaciones_parametros_out.values())
    puntuaciones_parametros_in_norm_iter = escalar_puntuaciones(puntuaciones_parametros.values())
        
    # Crear los DataFrames de puntuaciones de parámetros de salida
    puntuaciones_in_df = pd.DataFrame([puntuaciones_parametros])
    puntuaciones_out_df = pd.DataFrame([puntuaciones_parametros_out])
    puntuacion_total_df = pd.DataFrame([puntuaciones_iteraciones_TOTAL])
    puntuacion_total_df_transp = puntuacion_total_df.stack().reset_index(drop=True)
    puntuaciones_parametros_in_norm_df = pd.DataFrame([puntuaciones_parametros_in_norm_iter], columns=nombres_columnas_in)
    puntuaciones_parametros_out_norm_df = pd.DataFrame([puntuaciones_parametros_out_norm_iter], columns=nombres_columnas_out)
    
    # Crear los DataFrames donde se van agrupando las iteraciones
    df1_aux = pd.concat([df1_aux, df1])
    df2_aux = pd.concat([df2_aux, df2])
    puntuaciones_in_df_aux = pd.concat([puntuaciones_in_df_aux, puntuaciones_in_df])
    puntuaciones_out_df_aux = pd.concat([puntuaciones_out_df_aux, puntuaciones_out_df])
    puntuaciones_parametros_in_norm_df_aux = pd.concat([puntuaciones_parametros_in_norm_df_aux, puntuaciones_parametros_in_norm_df])
    puntuaciones_parametros_out_norm_df_aux = pd.concat([puntuaciones_parametros_out_norm_df_aux, puntuaciones_parametros_out_norm_df])

# Escalar/normalizar puntuaciones de las iteraciones y de los parámetros de entrada y de salida
nuevas_puntuaciones = escalar_puntuaciones(puntuaciones_iteraciones_TOTAL)
puntuacion_total_norm_df = pd.DataFrame([nuevas_puntuaciones])
puntuacion_total_norm_df_transp = puntuacion_total_norm_df.stack().reset_index(drop=True)

# Resetear los índices de los DataFrames
df2_aux.reset_index(drop=True, inplace=True)
puntuacion_total_df_transp.reset_index(drop=True, inplace=True)
# Agregado para el algoritmo de ia
df_ia = pd.concat([df_ia, df2_aux, puntuacion_total_norm_df_transp], axis=1)
df_ia.columns = nombres_columnas_in + ['Puntuación iteración']

# Definir la función para clasificar según la puntuación
def clasificar_puntuacion(puntuacion):
    if puntuacion >= 9:
        return 'Sobresaliente'
    elif puntuacion >= 7 and puntuacion <9:
        return 'Notable'
    elif puntuacion >= 5 and puntuacion <7:
        return 'Bien'
    elif puntuacion >= 3 and puntuacion < 5:
        return 'Deficiente'
    else:
        return 'Muy deficiente'

# Iterar sobre las filas de df_ia y clasificar la columna adicional de df_ia_clasif
df_ia_clasif = pd.DataFrame()
for index, row in df_ia.iterrows():
    puntuacion = row['Puntuación iteración']  # Obtener la puntuación de la fila actual
    etiqueta = clasificar_puntuacion(puntuacion)  # Clasificar la puntuación
    df_ia_clasif.at[index, 'Clasificación'] = etiqueta  # Asignar la etiqueta a la columna 20 de df_ia_clasif 

# Constuir df_ia_clasif añadiendo la clasificación
df_ia_clasif = pd.concat([df_ia, df_ia_clasif], axis=1)

# Escribir el DataFrame en el archivo Excel
df1_aux.to_csv(ruta_completa_in, index=False)
df2_aux.to_csv(ruta_completa_in, index=False)
puntuaciones_in_df_aux.to_csv(ruta_completa_marks_in, index=False)
puntuaciones_out_df_aux.to_csv(ruta_completa_marks_out, index=False)
puntuacion_total_df.to_csv(ruta_completa_marks, index=False)
puntuacion_total_norm_df_transp.to_csv(ruta_completa_marks_norm, index=False)
df_ia.to_csv(ruta_completa_ia, index=False)
df_ia_clasif.to_csv(ruta_completa_ia_clasif, index=False)

# Imprimir puntuaciones de todas las iteraciones
print("Puntuaciones de todas las iteraciones:")
for iteracion, puntuacion in enumerate(puntuaciones_iteraciones_TOTAL, start=1):
    print(f"Iteración {iteracion}: {puntuacion}")
    
# Imprimir los peores parámetros y sus puntuaciones de cada iteración
print("\nPeor parámetro y peor puntuación de cada iteración:")
for iteracion, (peor_parametro, peor_puntuacion) in enumerate(peores_parametros_puntuaciones_por_iteracion, start=1):
    print(f"Iteración {iteracion}: Peor parámetro '{peor_parametro}' con puntuación {peor_puntuacion}")

# Imprimir nuevas puntuaciones
print("\nNuevas puntuaciones:\n", nuevas_puntuaciones)
print("\nNuevas puntuaciones parámetros de entrada:\n", puntuaciones_parametros_in_norm_df_aux)
print("\nNuevas puntuaciones parámetros de salida:\n", puntuaciones_parametros_out_norm_df_aux)
print("\nArchivo para la regresión lineal:\n", df_ia)
print("\nArchivo para la clasificación:\n", df_ia_clasif)

Iteración 1:
mf: 1480.0823298589792
M: 0.4159222059128874
t: 294.6188797590221
p: 90950.65645617303
bpr: 9.756671994115775
PI_i: 1.0006142950300745
eta_fn: 0.9079693498775259
PI_fn: 1.4728980849813023
eta_c1: 0.9177119961105307
PI_c1: 1.6082845076416112
eta_c2: 0.9117130793114497
PI_c2: 9.699654239137432
eta_cc: 1.012942406178223
PI_cc: 0.9707555960389527
eta_t1: 0.8622015732652298
eta_t2: 0.9253195239085861
eta_n: 0.9873542591497869
LHV: 44052667.55049313
{'mf': 10.0, 'M': 5.0, 't': 9.0, 'p': 8.0, 'bpr': 9.0, 'PI_i': 8.0, 'eta_fn': 10.0, 'PI_fn': 9.0, 'eta_c1': 8.0, 'PI_c1': 6.0, 'eta_c2': 8.0, 'PI_c2': 8.0, 'eta_cc': 9.0, 'PI_cc': 7.0, 'eta_t1': 5.0, 'eta_t2': 8.0, 'eta_n': 9.0, 'LHV': 10.0}
{'mf': 1480.0823298589792, 'M': 0.4159222059128874, 't': 294.6188797590221, 'p': 90950.65645617303, 'bpr': 9.756671994115775, 'PI_i': 1.0006142950300745, 'eta_fn': 0.9079693498775259, 'PI_fn': 1.4728980849813023, 'eta_c1': 0.9177119961105307, 'PI_c1': 1.6082845076416112, 'eta_c2': 0.9117130793114

0.il     Inlet
          T0 = 288.713        [K]
          p0 = 100,910.275    [Pa]
0.fn     Fan
          T0 = 330.696        [K]
          p0 = 157,334.745    [Pa]
1.s.cp1  Compressor
          T0 = 381.983        [K]
          p0 = 249,796.975    [Pa]
1.s.cp2  Compressor
          T0 = 768.098        [K]
          p0 = 2,397,464.287  [Pa]
1.s.cc   Combustion chamber
          T0 = 1,561.371      [K]
          p0 = 2,446,382.175  [Pa]
1.s.tb1  Turbine
          T0 = 1,196.892      [K]
          p0 = 730,201.502    [Pa]
1.s.tb2  Turbine
          T0 = 714.715        [K]
          p0 = 67,523.672     [Pa]
2.nz     Nozzle
          T0 = 371.826        [K]
          p0 = 148,851.768    [Pa]
{'i.t0': 9.0, 'i.p0': 9.0, 'fn.t0': 8.0, 'fn.p0': 9.0, 'c1.t0': 8.0, 'c1.p0': 8.0, 'c2.t0': 7.0, 'c2.p0': 9.0, 'cc.t0': 9.0, 'cc.p0': 9.0, 't1.t0': 9.0, 't1.p0': 7.0, 't2.t0': 7.0, 't2.p0': 10.0, 'n.t0': 7.0, 'n.p0': 10.0}
{'i.t0': 288.71335077382696, 'i.p0': 100910.27538398575, 'fn.t0': 330.696462821

0.il     Inlet
          T0 = 288.757        [K]
          p0 = 93,654.459     [Pa]
0.fn     Fan
          T0 = 333.812        [K]
          p0 = 148,044.075    [Pa]
1.s.cp1  Compressor
          T0 = 380.438        [K]
          p0 = 219,757.470    [Pa]
1.s.cp2  Compressor
          T0 = 772.974        [K]
          p0 = 2,251,049.406  [Pa]
1.s.cc   Combustion chamber
          T0 = 1,615.004      [K]
          p0 = 2,296,560.607  [Pa]
1.s.tb1  Turbine
          T0 = 1,250.634      [K]
          p0 = 686,753.062    [Pa]
1.s.tb2  Turbine
          T0 = 723.414        [K]
          p0 = 60,846.639     [Pa]
2.nz     Nozzle
          T0 = 376.848        [K]
          p0 = 139,545.983    [Pa]
{'i.t0': 9.0, 'i.p0': 10.0, 'fn.t0': 7.0, 'fn.p0': 10.0, 'c1.t0': 8.0, 'c1.p0': 6.0, 'c2.t0': 7.0, 'c2.p0': 10.0, 'cc.t0': 8.0, 'cc.p0': 10.0, 't1.t0': 10.0, 't1.p0': 8.0, 't2.t0': 7.0, 't2.p0': 9.0, 'n.t0': 8.0, 'n.p0': 9.0}
{'i.t0': 288.75711227164254, 'i.p0': 93654.45932032526, 'fn.t0': 333.8122038

Puntuación iteración IN:  7.444
Puntuación iteración OUT:  8.5
Iteración 33:
mf: 1500.8837797720837
M: 0.38539857727064225
t: 270.3160576099918
p: 90068.83328466324
bpr: 9.204572174118947
PI_i: 0.9407125924100184
eta_fn: 0.9285981696549823
PI_fn: 1.599869916278895
eta_c1: 0.8771014612057053
PI_c1: 1.4970103819405487
eta_c2: 0.9310190781678809
PI_c2: 9.65929540569253
eta_cc: 0.9294668243577991
PI_cc: 0.9573377048201192
eta_t1: 0.8709154463624695
eta_t2: 0.8532068653039877
eta_n: 0.9655737625445641
LHV: 42060400.864472866
{'mf': 10.0, 'M': 5.0, 't': 7.0, 'p': 7.0, 'bpr': 10.0, 'PI_i': 8.0, 'eta_fn': 9.0, 'PI_fn': 8.0, 'eta_c1': 6.0, 'PI_c1': 6.0, 'eta_c2': 9.0, 'PI_c2': 8.0, 'eta_cc': 8.0, 'PI_cc': 6.0, 'eta_t1': 6.0, 'eta_t2': 5.0, 'eta_n': 9.0, 'LHV': 9.0}
{'mf': 1500.8837797720837, 'M': 0.38539857727064225, 't': 270.3160576099918, 'p': 90068.83328466324, 'bpr': 9.204572174118947, 'PI_i': 0.9407125924100184, 'eta_fn': 0.9285981696549823, 'PI_fn': 1.599869916278895, 'eta_c1': 0.87710146

Iteración 46:
mf: 1491.1630510789646
M: 0.4059750463030393
t: 279.5127371347137
p: 86541.59542432294
bpr: 9.787572355034007
PI_i: 0.9424343846407632
eta_fn: 0.8991585493842155
PI_fn: 1.4641370406444885
eta_c1: 0.9183529511936159
PI_c1: 1.5303159169841505
eta_c2: 0.8934515084168536
PI_c2: 9.520888333852886
eta_cc: 0.9230569978705904
PI_cc: 0.9393168738951563
eta_t1: 0.892749307284631
eta_t2: 0.8664105127193469
eta_n: 0.988460578844477
LHV: 45140474.764383554
{'mf': 10.0, 'M': 5.0, 't': 8.0, 'p': 7.0, 'bpr': 9.0, 'PI_i': 8.0, 'eta_fn': 10.0, 'PI_fn': 9.0, 'eta_c1': 8.0, 'PI_c1': 6.0, 'eta_c2': 7.0, 'PI_c2': 9.0, 'eta_cc': 8.0, 'PI_cc': 5.0, 'eta_t1': 7.0, 'eta_t2': 5.0, 'eta_n': 9.0, 'LHV': 10.0}
{'mf': 1491.1630510789646, 'M': 0.4059750463030393, 't': 279.5127371347137, 'p': 86541.59542432294, 'bpr': 9.787572355034007, 'PI_i': 0.9424343846407632, 'eta_fn': 0.8991585493842155, 'PI_fn': 1.4641370406444885, 'eta_c1': 0.9183529511936159, 'PI_c1': 1.5303159169841505, 'eta_c2': 0.893451508416

...

Puntuación iteración IN:  7.556
Puntuación iteración OUT:  8.5
Iteración 975:
mf: 1487.9383460427348
M: 0.4035484283348165
t: 272.8457236340154
p: 88742.12292976736
bpr: 9.662978322129915
PI_i: 1.0050337990797333
eta_fn: 0.911274472146685
PI_fn: 1.5168486856618866
eta_c1: 0.9300982794060086
PI_c1: 1.5617176764991327
eta_c2: 0.8968819186702005
PI_c2: 10.124556408515264
eta_cc: 0.9517080121858297
PI_cc: 0.9627219669925521
eta_t1: 0.8684036246363287
eta_t2: 0.8591929364467265
eta_n: 0.9413667987257444
LHV: 42452691.1074931
{'mf': 10.0, 'M': 5.0, 't': 7.0, 'p': 7.0, 'bpr': 9.0, 'PI_i': 8.0, 'eta_fn': 9.0, 'PI_fn': 8.0, 'eta_c1': 9.0, 'PI_c1': 6.0, 'eta_c2': 7.0, 'PI_c2': 8.0, 'eta_cc': 9.0, 'PI_cc': 7.0, 'eta_t1': 5.0, 'eta_t2': 5.0, 'eta_n': 10.0, 'LHV': 9.0}
{'mf': 1487.9383460427348, 'M': 0.4035484283348165, 't': 272.8457236340154, 'p': 88742.12292976736, 'bpr': 9.662978322129915, 'PI_i': 1.0050337990797333, 'eta_fn': 0.911274472146685, 'PI_fn': 1.5168486856618866, 'eta_c1': 0.930098279

Iteración 981:
mf: 1473.2872017411669
M: 0.3926611963074933
t: 277.19824699852336
p: 85493.44368939122
bpr: 9.625917521111303
PI_i: 0.957095205566736
eta_fn: 0.9256462918566188
PI_fn: 1.5761489956960255
eta_c1: 0.9278107331634238
PI_c1: 1.6155088831117224
eta_c2: 0.8517817631828986
PI_c2: 9.36953219434755
eta_cc: 0.9968426922839707
PI_cc: 1.0147854103971938
eta_t1: 0.8757270517654725
eta_t2: 0.9036107760289134
eta_n: 0.939788119025633
LHV: 41417354.56961545
{'mf': 10.0, 'M': 5.0, 't': 8.0, 'p': 6.0, 'bpr': 9.0, 'PI_i': 8.0, 'eta_fn': 9.0, 'PI_fn': 8.0, 'eta_c1': 9.0, 'PI_c1': 6.0, 'eta_c2': 5.0, 'PI_c2': 9.0, 'eta_cc': 9.0, 'PI_cc': 9.0, 'eta_t1': 6.0, 'eta_t2': 7.0, 'eta_n': 10.0, 'LHV': 9.0}
{'mf': 1473.2872017411669, 'M': 0.3926611963074933, 't': 277.19824699852336, 'p': 85493.44368939122, 'bpr': 9.625917521111303, 'PI_i': 0.957095205566736, 'eta_fn': 0.9256462918566188, 'PI_fn': 1.5761489956960255, 'eta_c1': 0.9278107331634238, 'PI_c1': 1.6155088831117224, 'eta_c2': 0.8517817631828

Iteración 987:
mf: 1380.281165656846
M: 0.39752813254250285
t: 272.28273839383877
p: 88620.38781216109
bpr: 9.892808996516052
PI_i: 1.015853290351784
eta_fn: 0.9601507358917466
PI_fn: 1.502719474864315
eta_c1: 0.912281081636341
PI_c1: 1.494213973767031
eta_c2: 0.8507265143612363
PI_c2: 10.244277988988372
eta_cc: 1.012412857077728
PI_cc: 0.9903874425802934
eta_t1: 0.8459221381453561
eta_t2: 0.872779083370167
eta_n: 0.9608647266160129
LHV: 46324124.83081546
{'mf': 8.0, 'M': 5.0, 't': 7.0, 'p': 7.0, 'bpr': 9.0, 'PI_i': 8.0, 'eta_fn': 7.0, 'PI_fn': 8.0, 'eta_c1': 8.0, 'PI_c1': 6.0, 'eta_c2': 5.0, 'PI_c2': 8.0, 'eta_cc': 9.0, 'PI_cc': 8.0, 'eta_t1': 4.0, 'eta_t2': 6.0, 'eta_n': 10.0, 'LHV': 9.0}
{'mf': 1380.281165656846, 'M': 0.39752813254250285, 't': 272.28273839383877, 'p': 88620.38781216109, 'bpr': 9.892808996516052, 'PI_i': 1.015853290351784, 'eta_fn': 0.9601507358917466, 'PI_fn': 1.502719474864315, 'eta_c1': 0.912281081636341, 'PI_c1': 1.494213973767031, 'eta_c2': 0.8507265143612363, '

Iteración 994:
mf: 1487.1534923298063
M: 0.38010574286357285
t: 285.64131377138017
p: 87784.07860780224
bpr: 9.725270054113253
PI_i: 1.0125786494518745
eta_fn: 0.8956364730096971
PI_fn: 1.5054247672723793
eta_c1: 0.8879282941230666
PI_c1: 1.5831422314923782
eta_c2: 0.8912084578489329
PI_c2: 9.894830700102085
eta_cc: 0.9439377993566113
PI_cc: 0.9393975286302244
eta_t1: 0.9100187284393523
eta_t2: 0.8599329843027773
eta_n: 0.9493668412881364
LHV: 43600914.44362864
{'mf': 10.0, 'M': 5.0, 't': 9.0, 'p': 7.0, 'bpr': 9.0, 'PI_i': 8.0, 'eta_fn': 10.0, 'PI_fn': 8.0, 'eta_c1': 7.0, 'PI_c1': 6.0, 'eta_c2': 7.0, 'PI_c2': 8.0, 'eta_cc': 9.0, 'PI_cc': 5.0, 'eta_t1': 8.0, 'eta_t2': 5.0, 'eta_n': 10.0, 'LHV': 10.0}
{'mf': 1487.1534923298063, 'M': 0.38010574286357285, 't': 285.64131377138017, 'p': 87784.07860780224, 'bpr': 9.725270054113253, 'PI_i': 1.0125786494518745, 'eta_fn': 0.8956364730096971, 'PI_fn': 1.5054247672723793, 'eta_c1': 0.8879282941230666, 'PI_c1': 1.5831422314923782, 'eta_c2': 0.89120

Puntuaciones de todas las iteraciones:
Iteración 1: 8.324
Iteración 2: 8.206
Iteración 3: 8.147
Iteración 4: 8.147
Iteración 5: 7.824
Iteración 6: 8.206
Iteración 7: 7.912
Iteración 8: 7.912
Iteración 9: 8.147
Iteración 10: 8.176
Iteración 11: 7.882
Iteración 12: 7.971
Iteración 13: 7.882
Iteración 14: 8.118
Iteración 15: 7.941
Iteración 16: 7.912
Iteración 17: 8.206
Iteración 18: 8.265
Iteración 19: 8.0
Iteración 20: 8.088
Iteración 21: 8.235
Iteración 22: 8.294
Iteración 23: 7.971
Iteración 24: 8.0
Iteración 25: 7.971
Iteración 26: 8.147
Iteración 27: 8.029
Iteración 28: 8.176
Iteración 29: 8.294
Iteración 30: 8.265
Iteración 31: 8.029
Iteración 32: 7.941
Iteración 33: 7.735
Iteración 34: 8.118
Iteración 35: 8.147
Iteración 36: 7.941
Iteración 37: 7.971
Iteración 38: 8.235
Iteración 39: 8.324
Iteración 40: 8.353
Iteración 41: 8.265
Iteración 42: 8.088
Iteración 43: 8.206
Iteración 44: 7.971
Iteración 45: 8.294
Iteración 46: 7.706
Iteración 47: 8.176
Iteración 48: 8.059
Iteración 49: 